In [1]:
import smartdata as smdt

In [2]:
import smartdata
from smartdata.utils.common import print_attributes as pa
from smartdata.utils.dframe import display_full_df as ddff
from smartdata.utils.common import to_namedtuple
from smartdata.utils.file import read_from_json
from smartdata import SmartData

In [3]:
import pandas as pd

In [4]:
from datetime import date, datetime

***

In [5]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline

In [6]:
import sys, os
import numpy as np
sys.path.append(r'C:\dev\code\gmt-amp\python-analytics-platform')
sys.path.append(r'C:\dev\code\gmt-amp\amp-models')
sys.path.append(r"C:\git\CHRIS_SCRIPTS")

# BENCHMARK

In [7]:
from smartdata.utils.log import Logger

In [8]:
Logger().switch_on('DEBUG')

In [9]:
myData = SmartData(empty=True)

In [10]:
myData.add_collection('benchmark')

In [11]:
# root_folder = r'C:\git\ARCHIVE\CHRIS_SCRIPTS\Full Run with price gateway (2years history)'
root_folder = r'Y:\gasmodels\net_back\run\2021_01_25'

In [12]:
myData.acquire.from_folder(root_folder,
                            date_parser=lambda col: pd.to_datetime(col, format='%d/%m/%Y'),
                            infer_datetime_format=True)

It took 0.950461 seconds to read dataFrame from file Algeria_Pivot_FullShip.csv
It took 1.45271 seconds to read dataFrame from file Algeria_Pivot_VarShip.csv
It took 1.34623 seconds to read dataFrame from file Angola_Pivot_FullShip.csv
It took 0.862448 seconds to read dataFrame from file Angola_Pivot_VarShip.csv
It took 0.751016 seconds to read dataFrame from file Cove point_Pivot_FullShip.csv
It took 1.44082 seconds to read dataFrame from file Cove point_Pivot_VarShip.csv
It took 1.00412 seconds to read dataFrame from file Gladstone_Pivot_FullShip.csv
It took 0.777714 seconds to read dataFrame from file Gladstone_Pivot_VarShip.csv
It took 137.828 seconds to read dataFrame from file Netback.csv
It took 55.3282 seconds to read dataFrame from file Netback_narrow.csv
It took 0.866017 seconds to read dataFrame from file Nigeria_Pivot_FullShip.csv
It took 0.78619 seconds to read dataFrame from file Nigeria_Pivot_VarShip.csv
It took 0.769151 seconds to read dataFrame from file Qatar_Pivot_Fu

In [13]:
myData.benchmark.df_Netback.shape[0]

1302420

In [14]:
#1300880 (22/01/2021)

## CODE

In [15]:
myData.add_collection('code')

In [16]:
run_folder = r'Y:\gasmodels\net_back\mongo_run'

In [17]:
myData.acquire.from_folder(run_folder, date_parser=lambda col: pd.to_datetime(col).date(), parse_dates=True, 
                           infer_datetime_format=True)

It took 0.778819 seconds to read dataFrame from file Algeria_Pivot_FullShip.csv
It took 0.84865 seconds to read dataFrame from file Algeria_Pivot_VarShip.csv
It took 0.749742 seconds to read dataFrame from file Angola_Pivot_FullShip.csv
It took 0.744699 seconds to read dataFrame from file Angola_Pivot_VarShip.csv
It took 0.749185 seconds to read dataFrame from file Cove point_Pivot_FullShip.csv
It took 0.862951 seconds to read dataFrame from file Cove point_Pivot_VarShip.csv
It took 0.710857 seconds to read dataFrame from file Gladstone_Pivot_FullShip.csv
It took 0.999886 seconds to read dataFrame from file Gladstone_Pivot_VarShip.csv
It took 143.33 seconds to read dataFrame from file Netback.csv
It took 58.7598 seconds to read dataFrame from file Netback_narrow.csv
It took 0.934537 seconds to read dataFrame from file Nigeria_Pivot_FullShip.csv
It took 1.23466 seconds to read dataFrame from file Nigeria_Pivot_VarShip.csv
It took 1.34557 seconds to read dataFrame from file Qatar_Pivot_F

## Reconcile

In [18]:
from datetime import datetime

In [19]:
def conv_dates_df(df, flag):
    if flag:
        df['PointDate_Origin'] = df['PointDate_Origin'].apply(lambda t: datetime.strptime(t, '%d/%m/%Y'))
        df['EffectiveDate'] = df['EffectiveDate'].apply(lambda t: datetime.strptime(t, '%d/%m/%Y'))
        df['PointDate_Destination'] = df['PointDate_Destination'].apply(lambda t: datetime.strptime(t, '%d/%m/%Y'))
    df['PointDate_Origin'] = df['PointDate_Origin'].apply(lambda t: pd.to_datetime(t).date())
    df['EffectiveDate'] = df['EffectiveDate'].apply(lambda t: pd.to_datetime(t).date())
    df['PointDate_Destination'] = df['PointDate_Destination'].apply(lambda t: pd.to_datetime(t).date())
    return df

In [20]:
all_passed = True
for name in sorted(list(set(set(myData.benchmark.names).union(myData.code.names)))):
    print(f'name={name} equal>>>>> ', end=' ')
    benchmark = myData.benchmark.get(name)
    benchmark = benchmark[benchmark['EffectiveDate']<'2021-01-18']
    benchmark = benchmark.reset_index(drop=True)
    try:
        benchmark = benchmark.drop('Unnamed: 0', axis=1)
    except:
        pass
    code = myData.code.get(name)    

    try:
        code = code[code['EffectiveDate']<'2021-01-18']
        code = code.reset_index(drop=True)
        pd.testing.assert_frame_equal(benchmark.transform(np.sort), code[benchmark.columns].transform(np.sort))
        print('PASSED')
    except Exception as e:        
        all_passed = False
        print('FAILED', end='')        
        print(e.args[0])
        break

name=df_Algeria_Pivot_FullShip equal>>>>>  PASSED
name=df_Algeria_Pivot_VarShip equal>>>>>  PASSED
name=df_Angola_Pivot_FullShip equal>>>>>  PASSED
name=df_Angola_Pivot_VarShip equal>>>>>  PASSED
name=df_Cove_point_Pivot_FullShip equal>>>>>  PASSED
name=df_Cove_point_Pivot_VarShip equal>>>>>  PASSED
name=df_Gladstone_Pivot_FullShip equal>>>>>  PASSED
name=df_Gladstone_Pivot_VarShip equal>>>>>  PASSED
name=df_Netback equal>>>>>  PASSED
name=df_Netback_narrow equal>>>>>  PASSED
name=df_Nigeria_Pivot_FullShip equal>>>>>  PASSED
name=df_Nigeria_Pivot_VarShip equal>>>>>  PASSED
name=df_Qatar_Pivot_FullShip equal>>>>>  PASSED
name=df_Qatar_Pivot_VarShip equal>>>>>  PASSED
name=df_Sabine_Pivot_FullShip equal>>>>>  PASSED
name=df_Sabine_Pivot_VarShip equal>>>>>  PASSED
name=df_T&T_Pivot_FullShip equal>>>>>  PASSED
name=df_T&T_Pivot_VarShip equal>>>>>  PASSED
name=df_Wheatstone_Pivot_FullShip equal>>>>>  PASSED
name=df_Wheatstone_Pivot_VarShip equal>>>>>  PASSED
name=df_Yamal_Pivot_FullShip equ

In [23]:
all_passed

True